### Задача

От заказчика передан пример выгрузки из канала: [ссылка на пример выгрузки](https://disk.yandex.ru/d/NcyC_Xp0EyE8ow).  
Необходимо распарсить JSON и преобразовать его в структуру `pandas DataFrame`.  
Пример ожидаемого результата: [ссылка на пример результата](https://drive.google.com/file/d/1Ry4zMi91-qqUVLds2OVCmmG95kH8gllt/view?usp=sharing).  

Каждый участник создает свою ветку и пушит код парсинга.

---

### Шаги решения

1. **Распарсить JSON**:
   - Загрузить JSON из файла.
   - Преобразовать его в Python-объект (словарь или список).

2. **Извлечь данные из вложенных структур**:
   - Извлечь основные данные о канале (`name`, `type`, `id`).
   - Обработать список сообщений (`messages`):
     - Для каждого сообщения извлечь основные данные (`id`, `type`, `date`, `from`, `from_id`).
     - Если текст сообщения представлен в виде списка, объединить его в одну строку.
     - Если есть реакции, обработать их и объединить в строку.

3. **Создать DataFrame**:
   - Объединить данные о канале и сообщениях.
   - Создать `pandas DataFrame`.

4. **Сохранить в CSV**:
   - Сохранить результат в CSV-файл.



In [6]:
import json
import pandas as pd

# Загрузка JSON из файла
with open('result.json', 'r', encoding='utf-8') as file:
    parsed_data = json.load(file)

# Извлечение данных о канале
channel_data = {
    "name": parsed_data["name"],
    "type": parsed_data["type"],
    "id": parsed_data["id"]
}

# Извлечение данных о сообщениях
messages = parsed_data["messages"]

# Создание списка для хранения данных
data = []

# Обработка каждого сообщения
for message in messages:
    # Базовые данные сообщения
    message_data = {
        "message_id": message["id"],
        "message_type": message["type"],
        "date": message["date"],
        "date_unixtime": message["date_unixtime"],
        "from": message["from"],
        "from_id": message["from_id"]
    }

    # Обработка текста сообщения (если есть)
    if "text" in message:
        # Если текст — это список, объединяем его в строку
        if isinstance(message["text"], list):
            text_parts = []
            for part in message["text"]:
                if isinstance(part, dict):
                    text_parts.append(part.get("text", ""))
                else:
                    text_parts.append(str(part))
            message_data["text"] = " ".join(text_parts)
        else:
            message_data["text"] = message["text"]

    # Обработка реакций (если есть)
    if "reactions" in message:
        reactions = []
        for reaction in message["reactions"]:
            reactions.append(f"{reaction['emoji']} ({reaction['count']})")
        message_data["reactions"] = ", ".join(reactions)

    # Добавление данных в общий список
    data.append({**channel_data, **message_data})

# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение в CSV
df.to_csv('output.csv', index=False, encoding='utf-8')

# Вывод первых строк DataFrame
df.head()

,name,type,id,message_id,message_type,date,date_unixtime,from,from_id,text,reactions
0,MANAGEMENT ALUMNI PRACTICUM,private_channel,1586396652,856,message,2025-02-01T14:06:16,1738407976,MANAGEMENT ALUMNI PRACTICUM,channel1586396652,"💥 💥 💥 💥 января 👋 \n\nПока мы вспоминали, ...","👍 (6), ❤ (2)"
1,MANAGEMENT ALUMNI PRACTICUM,private_channel,1586396652,857,message,2025-02-02T14:07:55,1738494475,MANAGEMENT ALUMNI PRACTICUM,channel1586396652,💭 Переговорный клуб \n\n→ 6 февраля в 20:00 ...,"🔥 (8), ❤ (2)"
2,MANAGEMENT ALUMNI PRACTICUM,private_channel,1586396652,858,message,2025-02-03T14:04:43,1738580683,MANAGEMENT ALUMNI PRACTICUM,channel1586396652,У Практикума новый знак ➡️ у выпускников нов...,"🔥 (11), ❤ (4), 😐 (1)"
3,MANAGEMENT ALUMNI PRACTICUM,private_channel,1586396652,859,message,2025-02-04T19:02:42,1738684962,MANAGEMENT ALUMNI PRACTICUM,channel1586396652,Прямой эфир с Олей Ладошкиной «Лидерство в 202...,"👍 (8), ❤ (4), 🔥 (3)"
4,MANAGEMENT ALUMNI PRACTICUM,private_channel,1586396652,860,message,2025-02-07T14:07:55,1738926475,MANAGEMENT ALUMNI PRACTICUM,channel1586396652,"Питчинг: как презентовать идею, чтобы её услыш...","🔥 (11), 👍 (4)"
